This file will create a speech recognition, too which someone can talk into their microphone, and the speech will be transcripted into text. 

Need to perform 2 tasks at the same time; one too continiously record the microphone, and the other too transcribe word too text. Way we do this is through threads (functions which run in the background)

In [1]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()

#Setting up widget for recording 
record_button = widgets.Button(
    description = 'Record',
    disabled = False,
    button_style = 'success',
    icon = 'microphone'
)

#Setting up widget for stopping the recording 
stop_button = widgets.Button(
    description = 'Stop',
    disabled = False,
    button_style = 'warning',
    icon = 'stop'
)

#Widget too show transcript as it's generated
output = widgets.Output()

#Function for when we start recording
def start_recording(data):
    messages.put(True)
    
    with output:
        display('Starting...')
        record = Thread(target = record_microphone)
        record.start()
        
        transcribe = Thread(target = speech_recognition, args = (output,))
        transcribe.start()

#Function for when we stop recording
def stop_recording(data):
    with output:
        messages.get()
        display('Stopped.')


#Linking widgets too functions above
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [4]:
#Need to figure out which of our sound devices we woud like too use
import pyaudio

#Initalize pyaudio connection too system audio devices
p = pyaudio.PyAudio()
#Loop through too print and see all of our audio devices
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate() #Terminate pyaudio connection to audio devices

{'index': 0, 'structVersion': 2, 'name': 'BlackHole 2ch', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.0014512471655328798, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.011609977324263039, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'MacBook Pro Microphone', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.03414965986394558, 'defaultLowOutputLatency': 0.01, 'defaultHighInputLatency': 0.04430839002267574, 'defaultHighOutputLatency': 0.1, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'MacBook Pro Speakers', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.01, 'defaultLowOutputLatency': 0.009229024943310658, 'defaultHighInputLatency': 0.1, 'defaultHighOutputLatency': 0.019387755102040816, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microso

In [5]:
#Constants too define how our audio will be recorded
CHANNELS = 1
FRAME_RATE = 16000 #How high quality recording is
RECORD_SECONDS = 20 #How many seconds we want too record audio for before transcription
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk = 1024): #chunk defines how often we read from microphone
    p = pyaudio.PyAudio()
    
    #Going too connect too our microphone and record
    stream = p.open(format = AUDIO_FORMAT,
                    channels = CHANNELS,
                    rate = FRAME_RATE,
                    input = True,
                    input_device_index = 1,
                    frames_per_buffer = chunk)
    
    frames = [] #Store all of our audio
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        #Setting up so every 20 seconds will send our audio data too transcription, then start new collection of audio
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = [] #Need to empty frames once for next chunk of audio data coming in
    
    #Need too stop and close connection once we're done recording
    stream.stop_stream()
    stream.close()
    p.terminate()